In [1]:
import json
import pandas as pd
import glob
import time
from tqdm import tqdm
import sys,os
import collections
import re
import csv
from datetime import datetime

In [2]:
# project = 'he'
# project = 'pp'
project = 'b'

if project == 'he':
    root_dir = '/Data/herbatica_365d/'
    session_file = 'sessions_all_he.csv'
    buyers_file = 'sessions_purchase_all_he.csv'
    session_buyers = "sessions_buyers_all_he.csv"
    folder = 'csv_he'

elif project == 'pp':
    root_dir = '/Data/particlepeptides_365d/'
    session_file = 'sessions_all_pp_500.csv'
    buyers_file = 'sessions_purchase_all_pp_500.csv'
    session_buyers = "sessions_buyers_all_pp_500.csv"
    folder = 'csv_pp_500'

elif project == 'b':
    root_dir = '/Data/barefootky_365d/'
    session_file = 'sessions_all_b_500.csv'
    buyers_file = 'sessions_purchase_all_b_500.csv'
    session_buyers = "sessions_buyers_all_b_500.csv"
    folder = 'csv_b_500'

In [3]:
data_file = 'data.json'

In [4]:
# load all files
listOfFiles = list()
with open(session_file, newline = '') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        listOfFiles.append(row[0])

In [5]:
# load purchase sessions
listOfBuyers = list()
with open(buyers_file, newline = '') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        listOfBuyers.append(row[0])

In [6]:
session_excludes = [ 
    'ip',
    'id',
    'isEu', 
    'latitude', 
    'longitude', 
    'projectId', 
    'useragent', 
    'version',
    'engine',
    'pageviews',
]
session_normalized_excludes = [
    'browser.family.name', 
    'browser.family.version', 
    'browser.type', 
    'browser.version',
    'browser.using.name',
    'browser.using.version',
    'device.manufacturer', 
    'device.model',
    'device.subtype',
    'device.series',
    'os.alias',
    'os.version',
    'os.version.value',
    'os.version.alias',
    'os.version.nickname',
    'os.edition',
    'os.family',
    'user.projectId',
    'user.number',
    'user.recorded'
]
pageview_excludes = [
    'screenWidth',
    'screenHeight',
    'width',
    'height',
]

In [7]:
df_sessions = pd.DataFrame()
df_users = pd.DataFrame()
df_pageviews = pd.DataFrame()

df_load = pd.DataFrame()
df_click = pd.DataFrame()
df_input = pd.DataFrame()
df_mouse_move = pd.DataFrame()
df_mouse_click = pd.DataFrame()
df_scroll_move = pd.DataFrame()
df_scrandom = pd.DataFrame()
df_wild_mouse = pd.DataFrame()
df_rage_click = pd.DataFrame()

In [8]:
action_types = ["input","load","click","mouse-click","rage-click","scroll-move","mouse-move","wild-mouse","scrandom"]

df_includes = {
    'input_includes':["pageview_id","session_id","id","type","at","args.duration","args.type","args.name"],
    'load_includes':["pageview_id","session_id","id","type","at",'args.firstInsertAt','args.domContentLoaded','args.firstPaint',
                   'args.firstContentfulPaint','args.loadEvent','args.firstInputDelay','args.timeToInteractive'],
    'click_includes':["pageview_id","session_id","id","type","at","args.double","args.type","args.text","args.position.x","args.position.y","args.target.left",
                    "args.target.top","args.target.width",'args.target.height'],
    'mouse-click_includes':["pageview_id","session_id","id","type","at","args.button","args.down.x","args.down.y","args.down.at","args.up.x","args.up.y",
                          "args.up.at","args.pause","args.duration","args.distance"],
    'rage-click_includes':["pageview_id","session_id","id","type","at","args.duration","args.center.x","args.center.y","args.intensity"],
    'scroll-move_includes':["pageview_id","session_id","id","type","at","args.duration","args.distance","args.id","args.source.pageLeft","args.source.pageTop","args.source.at",
                          "args.destination.pageLeft","args.destination.pageTop","args.destination.at","args.direction","args.distance2",
                          "args.vx","args.vy","args.v","args.a","args.jerk"],
    'mouse-move_includes':["pageview_id","session_id","id","type","at","args.duration","args.distance","args.source.at","args.destination.at","args.direction","args.distance2",
                         "args.vx","args.vy","args.v","args.a","args.jerk","args.source.x","args.source.y","args.destination.x",
                         "args.destination.y","args.straightness","args.jitter"],
    'wild-mouse_includes':["pageview_id","session_id","id","type","at","args.duration"],
    'scrandom_includes':["pageview_id","session_id","id","type","at","args.duration"]   
}

In [9]:
# list of buyers - user id
listOfUserBuyers = list()
for filename in listOfBuyers:
    with open(filename, 'r', encoding="utf8") as file:
        file = file.read()
        json_data = json.loads(file)           
        listOfUserBuyers.append(json_data['user']['id'])

In [10]:
listOfUserBuyers = (set(listOfUserBuyers))

In [11]:
len(listOfUserBuyers)

6040

In [ ]:
# all sessions of buyers
listOfSessions = list()
for filename in listOfFiles:
    with open(filename, 'r', encoding="utf8") as file:
        file = file.read()
        json_data = json.loads(file)
        if json_data['user']['id'] in listOfUserBuyers:
            listOfSessions.append(filename)

In [ ]:
f = open(session_buyers, "w")
f.writelines(["%s\n" % item  for item in listOfSessions])
f.close()

In [12]:
# load all sessions of buyers
listOfSessions = list()
with open(session_buyers, newline = '') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        listOfSessions.append(row[0])

In [13]:
len(listOfSessions) 

17738

In [ ]:
# pageview parsing
for filename in listOfSessions:
    with open(filename, 'r', encoding="utf8") as file:
        file = file.read()
        json_data = json.loads(file)        
        parsed_pageviews = pd.DataFrame(json_data['pageviews'])
        parsed_pageviews = parsed_pageviews.drop(columns=pageview_excludes, axis=1, errors='ignore')
        parsed_pageviews['session_id'] = json_data['sessionId'] # add session_id to pageviews to connect pageview to a specific session  
        try:
            parsed_pageviews = parsed_pageviews[parsed_pageviews['baked'].str.len() > 0] # not empty baked only 
            if not parsed_pageviews.empty:
                # sessions
                session_data = {k: json_data[k] for k in set(list(json_data.keys())) - set(session_excludes)}
                session_data = pd.json_normalize(session_data) # split into columns
                session_data = session_data.drop(columns=session_normalized_excludes, axis=1, errors='ignore')                     
                df_sessions = df_sessions.append(session_data, ignore_index=True, sort=False) # session append

                # save baked from pageview into parsed_events
                parsed_events = pd.DataFrame(parsed_pageviews[['id', 'session_id', 'baked']])
                parsed_pageviews = parsed_pageviews.drop(columns=['baked']) 

                # parsing pageviews                
                df_pageviews = df_pageviews.append(parsed_pageviews, ignore_index=True, sort=False) # pageview append
                parsed_events.rename(columns={'id':'pageview_id'}, inplace=True)
                parsed_events = parsed_events.explode('baked') # explode baked objects into rows

                # explode baked
                for key in parsed_events.iloc[0]['baked'].keys():
                    parsed_events[key] = parsed_events.baked.apply(lambda b: b[key] if type(b) is dict else None)

                parsed_events = parsed_events.drop(columns=['baked', 'modifiers'])
                
                for type_name in action_types:
                    type_includes = df_includes[f'{type_name}_includes']

                    type_data = parsed_events[parsed_events.type == type_name]
                    type_data = type_data.join(pd.json_normalize(type_data.args.values).add_prefix('args.'))
                    type_data = type_data.drop(columns=['args','type'], axis=1, errors='ignore')
                    type_data = type_data[type_data.columns & type_includes]

                    if (type_name == 'load'):
                        df_load = df_load.append(type_data, ignore_index=True)
                    elif (type_name == 'input'):
                        df_input = df_input.append(type_data, ignore_index=True)
                    elif (type_name == 'click'):
                        df_click = df_click.append(type_data, ignore_index=True)
                    elif (type_name == 'mouse-click'):
                        df_mouse_click = df_mouse_click.append(type_data, ignore_index=True)
                    elif (type_name == 'rage-click'):
                        df_rage_click = df_rage_click.append(type_data, ignore_index=True)
                    elif (type_name == 'scroll-move'):
                        df_scroll_move = df_scroll_move.append(type_data, ignore_index=True) 
                    elif (type_name == 'mouse-move'):
                        df_mouse_move = df_mouse_move.append(type_data, ignore_index=True)
                    elif (type_name == 'wild-mouse'):
                        df_wild_mouse = df_wild_mouse.append(type_data, ignore_index=True)
                    elif (type_name == 'scrandom'):
                        df_scrandom = df_scrandom.append(type_data, ignore_index=True)                   

        except KeyError:
            print("Baked empty")

In [15]:
def convertMS(ms):
    return datetime.fromtimestamp(ms/1000.0)

In [16]:
# SPRACOVANIE - result: 
# referrer, started_at, day, session_id, country, city, region, continent, user_id, device.type, browser.name, os.name, started_at_time, 
# ses_id

In [17]:
df_sessions = df_sessions.rename(columns={'startedAt': 'started_at', 'sessionId': 'session_id', 'user.id': 'user_id'})
df_sessions['started_at_time'] = df_sessions['started_at'].apply(lambda ms: convertMS(ms))
df_sessions['ses_id'] = pd.factorize(df_sessions['session_id'])[0] + 1 # replace sessionID with numbers (1...N)

In [ ]:
# USERS

In [19]:
df_users = pd.DataFrame(df_sessions[['user_id', 'started_at', 'started_at_time']])
df_users['u_id'] = pd.factorize(df_users['user_id'])[0] + 1 

In [21]:
# session count for each user
sessions_count = pd.DataFrame(df_users['u_id'].value_counts()) # user_id count
sessions_count.rename(columns={'u_id': 'n_sessions'}, inplace=True) 
df_users.drop_duplicates('u_id', keep='first', inplace=True)

In [22]:
sessions_count.head(3)

,n_sessions
11,797
342,92
28,85


In [23]:
df_users = pd.merge(df_users, sessions_count, left_on='u_id', right_index=True, how='left')

In [24]:
df_users.sort_values('n_sessions', ascending = False).head(6)

,user_id,started_at,started_at_time,u_id,n_sessions
10,31dca311-9dd8-4db1-925f-a286ce799357,1582543374162,2020-02-24 12:22:54.162,11,797
407,fefac58c-ef7f-4e25-9c10-3c40f69c7764,1596230586234,2020-07-31 23:23:06.234,342,92
27,5b7e6d35-43d3-4e7c-a95d-e756934700f3,1582578593189,2020-02-24 22:09:53.189,28,85
534,c8d386f0-984c-4646-822b-98adf85ef83a,1582493561874,2020-02-23 22:32:41.874,434,76
44,93669b9a-602c-4158-9739-d01042fb562a,1582829851490,2020-02-27 19:57:31.490,43,73
370,423ae395-74bf-40de-bfc0-a0b49f6d1066,1582743527885,2020-02-26 19:58:47.885,311,72


In [25]:
# PAGEVIEWS - result: 
# pageview_id, url, referrer, started_at, duration, inactivity, session_id, started_at_time, pv_id, ses_id, user_id

In [26]:
df_pageviews = df_pageviews.rename(columns={'id': 'pageview_id', 'startedAt': 'started_at'})
df_pageviews['started_at_time'] = df_pageviews['started_at'].apply(lambda ms: convertMS(ms))
df_pageviews['pv_id'] = pd.factorize(df_pageviews['pageview_id'])[0] + 1
df_pageviews.head(4)

,pageview_id,url,referrer,started_at,duration,inactivity,session_id,started_at_time,pv_id
0,0004f120-8ae1-11ea-9430-c6018a4fe069,https://www.barefootky.sk/kategoria/d-d-step/,https://www.google.com/,1588250954495,15436.0,"[{'start': 0, 'end': 941}, {'start': 991, 'end...",00034370-8ae1-11ea-bfdf-5db6a8c2950f,2020-04-30 14:49:14.495,1
1,0009f440-571a-11ea-8815-af26f5f1e9f3,https://www.barefootky.sk/kategoria/aigle-berg...,https://www.barefootky.sk/kategoria/afelo/,1582557977178,8044.0,"[{'start': 0, 'end': 248}, {'start': 1420, 'en...",0008bbc0-571a-11ea-aa71-584d5b4aad40,2020-02-24 16:26:17.178,2
2,000ccbd0-7f72-11eb-ab14-a72f742b3eee,https://www.barefootky.sk/kategoria/nanga/,,1615141316383,11552.0,"[{'start': 0, 'end': 3167}, {'start': 3167, 'e...",000b1e20-7f72-11eb-b2a5-593b07bb16d9,2021-03-07 19:21:56.383,3
3,0014e000-5736-11ea-ac09-d0f4c664cbe9,https://www.barefootky.sk/kategoria/aigle-berg...,https://www.barefootky.sk/kategoria/aigle-berg...,1582570002204,15377.0,"[{'start': 0, 'end': 380}, {'start': 380, 'end...",0013f5a0-5736-11ea-bca9-cd9798308c77,2020-02-24 19:46:42.204,4


In [27]:
df_pageviews = pd.merge(
    df_pageviews,
    df_sessions[['session_id', 'ses_id', 'user_id']],
    on='session_id',
    how='left'
)

In [ ]:
# SPRACOVANIE INACTIVITIES

In [29]:
df_inactivities = pd.DataFrame(df_pageviews[['pageview_id','pv_id','session_id','ses_id','user_id','started_at', 'started_at_time','inactivity']])
df_pageviews = df_pageviews.drop(columns=['inactivity'], axis=1, errors='ignore')
df_inactivities = df_inactivities.explode('inactivity')

In [33]:
# inactivity explode
for key in df_inactivities.iloc[0]['inactivity'].keys():
    df_inactivities[key] = df_inactivities.inactivity.apply(lambda b: b[key] if type(b) is dict else None)

In [34]:
df_inactivities = df_inactivities.drop(columns=['inactivity'])
df_inactivities.rename(columns={'start': 'inactivity_start'}, inplace=True)
df_inactivities.rename(columns={'end': 'inactivity_end'}, inplace=True)

In [35]:
# sum of inactivities
df_inactivities['inactivity_sum'] = df_inactivities['inactivity_end'] - df_inactivities['inactivity_start']

In [36]:
# sum of inactivities for user
df_inactivities = df_inactivities.sort_values(by=['user_id', 'started_at'])
df_inactivities['user_total_inactivity'] = df_inactivities.groupby('user_id')['inactivity_sum'].expanding(1).sum().values
df_inactivities['user_last_total_inactivity'] = df_inactivities.groupby('user_id')['inactivity_sum'].expanding(1).sum().values - df_inactivities['inactivity_sum']

In [37]:
# sum of inactivities for session
df_inactivities = df_inactivities.sort_values(by=['session_id', 'started_at'])
df_inactivities['session_total_inactivity'] = df_inactivities.groupby('session_id')['inactivity_sum'].expanding(1).sum().values
df_inactivities['session_last_total_inactivity'] = df_inactivities.groupby('session_id')['inactivity_sum'].expanding(1).sum().values - df_inactivities['inactivity_sum']

In [38]:
# sum of inactivities for pageview 
df_inactivities = df_inactivities.sort_values(by=['pageview_id', 'started_at']) 
df_inactivities['pageview_total_inactivity'] = df_inactivities.groupby('pageview_id')['inactivity_sum'].expanding(1).sum().values
df_inactivities['pageview_last_total_inactivity'] = df_inactivities.groupby('pageview_id')['inactivity_sum'].expanding(1).sum().values - df_inactivities['inactivity_sum']

In [40]:
# for each pageview compute effective_duration based on df_inactivities['pageview_total_inactivity']
df_inactivities = df_inactivities.sort_values(by=['pageview_id', 'started_at'])

df_pageviews = pd.merge(
    df_pageviews,
    df_inactivities[['pageview_id', 'pageview_total_inactivity']],
    how='left',
    on='pageview_id'
)

df_pageviews.drop_duplicates('pageview_id', keep='last', inplace=True) 
df_pageviews['effective_duration'] = df_pageviews['duration'] - df_pageviews['pageview_total_inactivity']
df_pageviews = df_pageviews.drop(columns=['pageview_total_inactivity'])

In [41]:
# for each session compute effective_duration based on df_pageviews['effective_duration']
df_pageviews = df_pageviews.sort_values(by=['session_id', 'started_at'])
df_sessions = df_sessions.sort_values(by=['session_id', 'started_at'])

df_sessions = pd.merge(
    df_sessions,
    df_pageviews[['session_id', 'effective_duration']],
    how='left',
    on='session_id'
)

df_sessions['pageviews_effective_duration'] = df_sessions.groupby('session_id')['effective_duration'].expanding(1).sum().values
df_sessions.drop_duplicates('session_id', keep='last', inplace=True)
df_sessions = df_sessions.drop(columns=['effective_duration'])
df_sessions = df_sessions.rename(columns={'pageviews_effective_duration': 'effective_duration'})

In [42]:
# for each session compute effective_time_from_start based on effective_duration sessions 
df_sessions = df_sessions.sort_values(by=['user_id', 'started_at'])
df_sessions['effective_time_from_start'] = df_sessions.groupby('user_id')['effective_duration'].expanding(1).sum().values - df_sessions['effective_duration']

In [43]:
# for each pageview compute effective_time_from_start based on effective_duration pageviews
df_pageviews = df_pageviews.sort_values(by=['user_id', 'started_at'])
df_pageviews['effective_time_from_start'] = df_pageviews.groupby('user_id')['effective_duration'].expanding(1).sum().values - df_pageviews['effective_duration']

In [44]:
# SPRACOVANIE BAKED

In [45]:
def add_data_from_pageview(dataframe):
    
    dataframe = pd.merge(
        dataframe,
        df_pageviews[['pageview_id', 'pv_id', 'user_id', 'effective_time_from_start']],
        on='pageview_id',
        how='left'
    )

    dataframe = dataframe.rename(columns={'effective_time_from_start': 'pageview_effective_time_from_start'})
    
    return dataframe

In [46]:
df_input = add_data_from_pageview(df_input)
df_load = add_data_from_pageview(df_load)
df_click = add_data_from_pageview(df_click)
df_mouse_click = add_data_from_pageview(df_mouse_click)
df_rage_click = add_data_from_pageview(df_rage_click)
df_mouse_move = add_data_from_pageview(df_mouse_move)
df_scroll_move = add_data_from_pageview(df_scroll_move)
df_wild_mouse = add_data_from_pageview(df_wild_mouse)
df_scrandom = add_data_from_pageview(df_scrandom)

In [47]:
# for all events compute effective_time_from_start based on pageviews duration and inactivities length
def count_effective_time_from_start(dataframe):

    dataframe = dataframe.reset_index().rename(columns={'index': 'event_id'})

    dataframe = pd.merge(
        dataframe,
        df_inactivities[['pageview_id', 'pv_id','inactivity_end', 'pageview_total_inactivity']],
        on=['pageview_id','pv_id'],
        how='left'
    )

    dataframe = dataframe[dataframe['inactivity_end'] <= dataframe['at']] 
    dataframe.drop_duplicates('event_id', keep='last', inplace=True) 
    dataframe['effective_time_from_start'] = dataframe['pageview_effective_time_from_start'] + dataframe['at'] - dataframe['pageview_total_inactivity']
    dataframe = dataframe.drop(columns=['inactivity_end', 'pageview_total_inactivity', 'pageview_effective_time_from_start'])
    dataframe = dataframe.set_index('event_id')
    
    return dataframe

In [48]:
df_input = count_effective_time_from_start(df_input)
df_load = count_effective_time_from_start(df_load)
df_click = count_effective_time_from_start(df_click)
df_mouse_click = count_effective_time_from_start(df_mouse_click)
df_rage_click = count_effective_time_from_start(df_rage_click)
df_mouse_move = count_effective_time_from_start(df_mouse_move)
df_scroll_move = count_effective_time_from_start(df_scroll_move)
df_wild_mouse = count_effective_time_from_start(df_wild_mouse)
df_scrandom = count_effective_time_from_start(df_scrandom)

In [50]:
df_inactivities.to_csv(folder + '/df_inactivities_all.csv', index=False, encoding='utf-8-sig')

df_input.to_csv(folder + '/df_input_all.csv', index=False, encoding='utf-8-sig')
df_load.to_csv(folder + '/df_load_all.csv', index=False, encoding='utf-8-sig')
df_click.to_csv(folder + '/df_click_all.csv', index=False, encoding='utf-8-sig')
df_mouse_click.to_csv(folder + '/df_mouse_click_all.csv', index=False, encoding='utf-8-sig')
df_rage_click.to_csv(folder + '/df_rage_click_all.csv', index=False, encoding='utf-8-sig')
df_mouse_move.to_csv(folder + '/df_mouse_move_all.csv', index=False, encoding='utf-8-sig')
df_scroll_move.to_csv(folder + '/df_scroll_move_all.csv', index=False, encoding='utf-8-sig')
df_wild_mouse.to_csv(folder + '/df_wild_mouse_all.csv', index=False, encoding='utf-8-sig')
df_scrandom.to_csv(folder + '/df_scrandom_all.csv', index=False, encoding='utf-8-sig')

df_pageviews.to_csv(folder + '/df_pageviews_all.csv', index=False, encoding='utf-8-sig')
df_sessions.to_csv(folder + '/df_sessions_all.csv', index=False, encoding='utf-8-sig')